**Definizione del grafo**

Analisi delle caratteristiche e della vulnerabilità della Rete di Trasporto Pubblica Metropolitana Milanese  
*Progetto del corso di Data Analytics*


Andrea Broccoletti  
*Matricola 886155*

In [16]:
import igraph as ig
import pandas as pd
import numpy as np

# Caricamento dei dati
Caricamento dei file csv, elaborazioni dei dataset GTFS. Le linee metropolitane e le linee di superficie risiedono in due file diversi.

In [17]:
underground_routes = pd.read_csv('../assets/static/gtfs/atm_underground_routes.csv')
underground_stops = pd.read_csv('../assets/static/gtfs/atm_underground_stops.csv')
underground_trips = pd.read_csv('../assets/static/gtfs/atm_underground_trips.csv')

## Pulizia preliminare dei dati e delle intestazioni
I file csv presentano intestazioni diverse e devono essere uniformati per poter procedere all'unione

In [18]:
underground_stops = underground_stops[['id_amat', 'nome', 'linee', 'LONG_X_4326', 'LAT_Y_4326']]
underground_stops.columns = ['stop_id', 'stop_name', 'lines', 'longitude', 'latitude']

underground_stops["lines"] = underground_stops["lines"].apply(lambda x: list(map(int, x.split(",")))) 

In [19]:
underground_trips = underground_trips[['percorso', 'num', 'id_ferm']]
underground_trips.columns = ['route_id', 'sequence', 'stop_id']

In [20]:
underground_routes = underground_routes[['linea', 'mezzo', 'percorso']]
underground_routes.columns = ['route_name', 'route_type', 'route_id']

# Costruzione del grafo
Costruzione del grafo non diretto in forma L-space, in cui i nodi sono stazioni e gli archi collegamenti di una linea.

In [21]:
G = ig.Graph(directed=False)

## Costruzione dei nodi
Ogni stazione corrisponde ad un nodo della rete.

In [22]:
# Stops incluse in almeno un trips
active_stops = underground_trips["stop_id"].unique()

In [23]:
for stop in active_stops:

    stop_info = underground_stops[underground_stops["stop_id"] == stop]
    
    if not stop_info.empty:
        G.add_vertex(name=stop, 
                     stop_name=stop_info["stop_name"].values[0], 
                     longitude=stop_info["longitude"].values[0], 
                     latitude=stop_info["latitude"].values[0], 
                     lines=stop_info["lines"].values[0])

## Costruzione degli archi
Ogni arco corrisponde ad un collegamento tra due stazioni successive. 
La presenza di un arco significa che almeno una linea collega le due stazioni (nodi) coinvolte.

In [24]:
for route in underground_trips["route_id"].unique():
 
    trips = underground_trips[underground_trips["route_id"] == route]
    route_info = underground_routes[underground_routes["route_id"] == route]
    
    trips = trips.sort_values(by="sequence")
    trip_stops = trips["stop_id"].tolist()

    for i in range(len(trip_stops) - 1):

        if G.are_adjacent(G.vs.find(name=np.int64(trip_stops[i])), 
                           G.vs.find(name=np.int64(trip_stops[i + 1]))):
            
            edge = G.es.find(_between=([G.vs.find(name=np.int64(trip_stops[i])).index], 
                                        [G.vs.find(name=np.int64(trip_stops[i + 1])).index]))
            
            # Conversione delle linee in liste di interi
            existing_lines = set(map(int, edge["lines"]))
            existing_lines.add(int(route_info["route_name"].values[0]))
            edge["lines"] = list(existing_lines)
            
        else:
            G.add_edge(G.vs.find(name=np.int64(trip_stops[i])), 
                       G.vs.find(name=np.int64(trip_stops[i + 1])), lines=[int(route_info["route_name"].values[0])])

# Modifica dei collegamenti
Alcune stazioni metropolitane presentano nomi diversi anche se sono fisicamente la stessa stazione, quindi devono essere rappresentate con lo stesso nodo.

In [25]:
renamed_stops = {"LORETO": ["LORETO M2", "LORETO M1"],
                 "CADORNA": ["CADORNA FN M1", "CADORNA FN M2"],
                 "DUOMO": ["DUOMO M1", "DUOMO M3"],
                 "LOTTO": ["LOTTO FIERAMILANOCITY", "LOTTO M5"],
                 "SAN AMBROGIO": ["S.AMBROGIO", "SAN AMBROGIO"]}

In [26]:
for new_name, old_names in renamed_stops.items():

    old_vertices = [v for v in G.vs if v['stop_name'] in old_names]
    if len(old_vertices) < 2:
        continue

    # creazione del nuovo nodo
    new_vertex = G.add_vertex(name=new_name)
    new_vertex['stop_name'] = new_name
    new_vertex['longitude'] = np.mean([v['longitude'] for v in old_vertices])
    new_vertex['latitude'] = np.mean([v['latitude'] for v in old_vertices])
    new_vertex['lines'] = list(set().union(*[v['lines'] for v in old_vertices]))

    # collegamento dei nuovi nodi ai neighbor dei vecchi nodi mantenendo gli attributi degli archi
    for old_vertex in old_vertices:
        neighbors = G.neighbors(old_vertex)

        for neighbor_index in neighbors:
            neighbor_vertex = G.vs[neighbor_index]

            if G.are_adjacent(new_vertex, neighbor_vertex):
                edge = G.es.find(_between=([new_vertex.index], [neighbor_vertex.index]))
                existing_lines = set(map(int, edge["lines"]))
                existing_lines.update(map(int, G.es.find(_between=([old_vertex.index], [neighbor_vertex.index]))["lines"]))
                edge["lines"] = list(existing_lines)
                
            else:
                old_edge = G.es.find(_between=([old_vertex.index], [neighbor_vertex.index]))
                G.add_edge(new_vertex, neighbor_vertex, lines=old_edge["lines"])

    # rimozione dei vecchi nodi
    G.delete_vertices([v.index for v in old_vertices]) 

# Salvataggio
Il grafo risultante viene salvato in formato Graphml perdendo gli attributi lista.  
Viene salvato anche il formato Pickle per poter essere usato nelle fasi successive

In [27]:
G.write_graphml('../assets/static/graphs/atm_network.graphml')

In [28]:
G.write_pickle('../assets/static/graphs/atm_network.pkl')